In [67]:
import pandas as pd
import numpy as np

Import all the files

In [68]:
visits = pd.read_csv('visits.csv', parse_dates=[1])
cart = pd.read_csv('cart.csv', parse_dates=[1])
checkout = pd.read_csv('checkout.csv', parse_dates=[1])
purchase = pd.read_csv('purchase.csv', parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [69]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [70]:
visits_cart = pd.merge(visits,cart,how='left')
print(visits_cart.head())

                                user_id          visit_time   
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00  \
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  


Step 3: How long is `visits_cart`?

In [71]:
num_rows_visit_cart = len(visits_cart)
print(num_rows_visit_cart)

2000


Step 4: How many timestamps are null for `cart_time`?

In [72]:
# method 1
null_cart_time = visits_cart['cart_time'].isna().sum()
print(null_cart_time)

# method 2
null_cart_time = len(visits_cart.loc[visits_cart['cart_time'].isnull()])
print(null_cart_time)

1652
1652


Step 5: What percentage only visited?

In [73]:
percent_visited = float(null_cart_time / len(visits_cart['cart_time']))
print(percent_visited)

0.826


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [74]:
visits_cart_checkout = pd.merge(visits_cart,checkout,how='left')
# print(visits_cart_checkout.head())

placed_in_cart_no_checkout = len(visits_cart_checkout.loc[(visits_cart_checkout['checkout_time'].isnull()) & visits_cart_checkout['cart_time'].notnull()]) 
placed_in_cart = len(visits_cart_checkout.loc[visits_cart_checkout['cart_time'].notnull()])
percent_cart_nocheckout =  placed_in_cart_no_checkout / placed_in_cart 

print(percent_cart_nocheckout)

0.3505747126436782


Step 7: Merge it all together

In [75]:
masterlist = pd.merge(visits_cart_checkout,purchase,how='left')
print(masterlist.head())

                                user_id          visit_time   
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00  \
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT                 NaT  
4                 NaT                 NaT                 NaT  


Step 8: % of users who got to checkout but did not purchase

In [81]:
users_at_checkout = len(masterlist.loc[masterlist['checkout_time'].notnull()])
users_at_checkout_notpurchased = len(masterlist.loc[(masterlist['purchase_time'].isnull()) & (masterlist['checkout_time'].notnull())])

percent_users_checkout_nopurchase = users_at_checkout_notpurchased / users_at_checkout
print(f"The % of users who got to checkout but did not purchase is: {percent_users_checkout_nopurchase}.")


The % of users who got to checkout but did not purchase is: 0.24550898203592814.


Step 9: check each part of the funnel, let's print all 3 of them again

In [85]:
print(f"The % of people who only visited are: {round(percent_visited,4)*100}.")
print(f"The % of people who placed their items in the cart but did not checkout is: {round(percent_cart_nocheckout,4)*100}.")
print(f"The % of people who got to the checkout but did not purchase is: {round(percent_users_checkout_nopurchase,4)*100}.")

The % of people who only visited are: 82.6.
The % of people who placed their items in the cart but did not checkout is: 35.06.
The % of people who got to the checkout but did not purchase is: 24.55.


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: Using the giant merged DataFrame masterlist that you created, let’s calculate the average time from initial visit to final purchase. Add a column that is the difference between purchase_time and visit_time.

In [90]:
masterlist['time_to_purchase'] = masterlist['purchase_time'] - masterlist['visit_time']


Step 11: examine the results

In [89]:
print(masterlist.head())

                                user_id          visit_time   
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00  \
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time time_to_purchase  
0                 NaT                 NaT                 NaT              NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  0 days 00:44:00  
2 2017-08-20 08:31:00                 NaT                 NaT              NaT  
3                 NaT                 NaT                 NaT              NaT  
4                 NaT                 NaT                 NaT              NaT  


Step 12: average time to purchase

In [88]:
avg_visit_to_purchase_time = masterlist.time_to_purchase.mean()

print(f"The average time from initial visit to final purchase is {avg_visit_to_purchase_time}.")

The average time from initial visit to final purchase is 0 days 00:43:12.380952380.
